In [1]:
import openai
import os

In [3]:
# get string of the key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
country = input("What country are you interested in for news?").lower()

In [5]:
country

'france'

For this we are limiting to Spain, France

In [6]:
country_newspapers = {"Spain".lower():"https://elpais.com/",
                      "France".lower():"https://www.lemonde.fr/"}

In [7]:
country_newspapers[country]

'https://www.lemonde.fr/'

In [8]:
import requests
import bs4

In [9]:
url = country_newspapers[country]

In [10]:
result = requests.get(url)

In [11]:
tag = "article__title-label"

In [12]:
soup = bs4.BeautifulSoup(result.text,'lxml')

In [13]:
soup.select('.article__title-label')

[<p class="article__title-label">En direct : pour les manifestants à Lyon, « Macron n’écoute pas, il se comporte comme un roi »</p>]

In [16]:
# other tags
tag = "article__title"
len(soup.select('.article__title'))
soup.select('.article__title')[:3]  # Top 3 headline

[<p class="article__title">Aux Etats-Unis, la Réserve fédérale poursuit sa hausse des taux, en dépit de la crise financière et du risque de récession</p>,
 <p class="article__title">Dans le sillage de ChatGPT, la course à l’intelligence artificielle</p>,
 <p class="article__title">« Le bilan de la Silicon Valley Bank n’était pas si mauvais, mais placé en obligations »</p>]

In [17]:
soup.select('.article__title')[0].getText()

'Aux Etats-Unis, la Réserve fédérale poursuit sa hausse des taux, en dépit de la crise financière et du risque de récession'

In [18]:
for item in soup.select('.article__title')[:3]:
    print(item.getText())

Aux Etats-Unis, la Réserve fédérale poursuit sa hausse des taux, en dépit de la crise financière et du risque de récession
Dans le sillage de ChatGPT, la course à l’intelligence artificielle
« Le bilan de la Silicon Valley Bank n’était pas si mauvais, mais placé en obligations »


In [28]:
# add tags to dict
# Country: (URL, TAG)
country_newspapers = {"Spain".lower():("https://elpais.com/",'.c_t '),
                      "France".lower():("https://www.lemonde.fr/",'.article__title-label')}

In [33]:
def create_news_prompt():
    country = input("What would you like a news summary for? ").lower()

    try:
        url,tag = country_newspapers[country]
    except:
        print("Sorry, country not supported")
        return
    
    #Scrape website
    results = requests.get(url)
    soup = bs4.BeautifulSoup(result.text,'lxml')

    # grab all the text
    country_headlines= ""
    for item in soup.select(tag)[:3]:
        country_headlines += item.getText() + "\n"

    prompt = "Detect the language of the news headlines, then translate a summary of the headlines to English in a conversational tone.\n"
    return prompt + country_headlines


In [36]:
prompt = create_news_prompt()
print(prompt)

Detect the language of the news headlines, then translate a summary of the headlines to English in a conversational tone.
En direct : pour les manifestants à Lyon, « Macron n’écoute pas, il se comporte comme un roi »



In [38]:
response = openai.Completion.create(
    model = 'text-davinci-003',
    prompt = prompt,
    max_tokens = 200,
    temperature = 0.1,
    top_p = 1.0
)

In [39]:
print(response['choices'][0]['text'])


The news headline is in French. A translation of the headline in a conversational tone would be: Protesters in Lyon are saying that President Macron is not listening to them and is acting like a king.
